In [1]:
import pyodbc
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("C:/Users/BoydClaire/.Renviron"))

True

In [2]:
def get_database_tables(database_name, connection_args, schema=None):

    connection_string = "".join(connection_args)

    # create connection object
    cnxn = pyodbc.connect(connection_string)

    cursor = cnxn.cursor()

    if schema:
        tables_raw = cursor.tables(schema=schema)
    else:
        tables_raw = cursor.tables()

    tables = pd.DataFrame(tables_raw)
    tables.columns = ['list']
    cleaned_tables = pd.DataFrame(tables['list'].apply(list).to_list(), columns = ['DATABASE_NAME','SCHEMA','NAME','TYPE','SELF_REFERENCING_COL_NAME'])
    cleaned_tables['db'] = database_name

    return cleaned_tables

In [3]:
# get sql tables

sql_database_list = ["production", "test", "sandbox"]
sql_tables = []

for database in sql_database_list:

    # build connection string
    connection_args = ["Driver={SQL Server};",
                        f"SERVER={os.environ[f'{database}_server']};",
                        f"DATABASE={os.environ[f'{database}_database']};",
                        f"UID={os.environ[f'{database}_username']};",
                        f"PWD={os.environ[f'{database}_password']};",
                        f"Trusted_Connection=no;",
                        "Port=1433"
    ]

    table = get_database_tables(database, connection_args)
    sql_tables.append(table)

In [4]:
# get oracle tables

oracle_database_list = ["fdw", "ias"]
oracle_tables = []

for database in oracle_database_list:

    # build connection string
    database_schema = os.getenv(f"{database}_schema")
    connection_args = ["Driver={Oracle in OraClient19Home1};",
                        f"DBQ={os.environ[f'{database}_path']};"
                        f"DATABASE={os.environ[f'{database}_schema']};"
                        f"UID={os.environ[f'{database}_username']};",
                        f"PWD={os.environ[f'{database}_password']};",
                        f"Trusted_Connection=no;",
                        "Port=1433"
    ]

    table = get_database_tables(database, connection_args)
    table = table.loc[table.loc[:,"SCHEMA"] == database_schema.upper(), :]

    oracle_tables.append(table)

In [5]:
flat = [table for tables in [sql_tables, oracle_tables] for table in tables]
all_tables = pd.concat(flat, axis=0)

In [6]:
all_tables.groupby("db").count()

,DATABASE,SCHEMA,NAME,TYPE,SELF_REFERENCING_COL_NAME
db,,,,,
fdw,0,228,228,228,0
ias,0,6471,6471,6471,0
production,4684,4684,4684,4684,0
sandbox,4265,4265,4265,4265,0
test,4550,4550,4550,4550,0


In [21]:
all_tables.drop(columns=["SELF_REFERENCING_COL_NAME"])

all_tables = all_tables 

all_tables.to_csv("data_dictionary_table_data.csv")

things to consider:
* repeats of the same data
* how often records get updated
* mirrored tables in different databases
* 


how to pull in cols for all databases based on the tables above
*

In [17]:
database= "test"

# build connection string
connection_args = ["Driver={SQL Server};",
                    f"SERVER={os.environ[f'{database}_server']};",
                    f"DATABASE={os.environ[f'{database}_database']};",
                    f"UID={os.environ[f'{database}_username']};",
                    f"PWD={os.environ[f'{database}_password']};",
                    f"Trusted_Connection=no;",
                    "Port=1433"
]

connection_string = "".join(connection_args)

# create connection object
cnxn = pyodbc.connect(connection_string)

cursor = cnxn.cursor()

def get_columns_from_table(cursor, tablename):
    
    cursor.execute(f"""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{tablename}'
    """)

    # Fetch all results
    columns_info = pd.DataFrame(cursor.fetchall())

    return columns_info


obj = get_columns_from_table(cursor, "PARCEL")

In [18]:
obj.head()

,0
0,"[PRC_MNC, numeric]"
1,"[PRC_PID, numeric]"
2,"[PRC_ASSNG_DIST, varchar]"
3,"[PRC_ASSNG_SUBDIST, varchar]"
4,"[PRC_RES_DIST, varchar]"
